In [83]:
%%file main.py
import io
import logging
import os
import struct
import numpy as np
import pandas as pd
import panel
from panel import widgets
import param
from scipy.sparse import csr_matrix, spmatrix
from modelforge.model import assemble_sparse_matrix, disassemble_sparse_matrix
from sqlalchemy import create_engine

from commit_time_series.data_export import deserialize_sparse_matrix
from commit_time_series.seriation import Seriator
from commit_time_series.extractor import Columns

import numpy
import pandas
from typing import List, NamedTuple, Tuple

class DataManager(param.Parameterized):
    file_input = widgets.FileInput(name="Parquet file")
    dbtype = widgets.TextInput(name="Database type", value="postgresql")
    user = widgets.TextInput(name="Username", value="superset")
    password = widgets.TextInput(name="Password", value="superset")
    host = widgets.TextInput(name="Host", value="localhost")
    endpoint = widgets.TextInput(name="Host", value="superset")
    port = widgets.LiteralInput(name="Port", value=5432)
    connect_btn = widgets.Button(name="Connect to DB")
    data_source = widgets.TextInput(name="Table")
    datasource_btn = widgets.Button(name="Query table")
    
    def __init__(self):
        super().__init__()
        self.df = None
        self.datasource = None
        self.engine = None
        self._connected = False
    
    @property
    def url(self):
        url = "{}://{}:{}@{}:{}/{}".format(self.dbtype.value, self.user.value,
                                           self.password.value, self.host.value,
                                           self.port.value, self.endpoint.value)
        return url
    
    @param.depends("connect_btn.clicks", watch=True)
    def connect_to_db(self):
        msg = panel.pane.Str("Connected to \n{}".format(self.url))
        if self.connect_btn.clicks == 0:
            return panel.pane.Str("Ready to connect to \n{}".format(self.url))
        if self._connected:
            return msg
        try:
            engine = create_engine(self.url)
            self.engine = engine
            self._connected = True
            return msg
        except Exception as e:
            self._connected = False
            return panel.pane.Str("Error connecting to database \n{}".format(self.url))
        return panel.pane.Str("Not Connected to any database \n{}".format(self.url))
    
    def panel(self):
        widgets = panel.Row(panel.Column(self.dbtype, self.host, self.port), 
                         panel.Column(self.user, self.password, self.endpoint),
                         panel.Column(self.connect_to_db, self.connect_btn))
        return panel.Column(panel.pane.Markdown("## Connect to superset database"),
                            widgets)
    
class CommitTimeSeriesPreprocessor(param.Parameterized):

    EMBEDDINGS_TABLE = "developer_embeddings"
    COMMITS_TABLE = "commit_series"
    GROUPED_COMMITS_TABLE = "commits_per_repo"
    GROUPED_EMBEDDINGS_TABLE = "clusters_per_repo"
    embeddings_file = widgets.FileInput(name="Embeddings")
    commits_file = widgets.FileInput(name="Commits")
    grouped_file = widgets.FileInput(name="Grouped series")
    load_parquet_btn = widgets.Button(name="Load files")
    to_sql_btn = widgets.Button(name="write to database.")

    def __init__(self, engine):
        from sqlalchemy import create_engine
       
        self.embeddings = None
        self.grouped = None
        self.commits = None
        self.engine = engine
        self._data_loaded = False

    @staticmethod
    def extract_commits_per_repo(df):
        """
        Extract a DataFrame with columns ['date_ix', 'commits', 'name', 'repository'] \
        from grouped commit series data.
        """
        def get_one_commit_series(x):
            data = pandas.DataFrame(index=df.loc[0, Columns.Value],
                                    data=numpy.array(x[Columns.Value].todense()).flatten(),
                                    columns=[Columns.Value])
            data.index.name = Columns.Date
            data = data.reset_index()
            data = data[data[Columns.Value] > 0]
            data[Columns.SeriesId] = x[Columns.SeriesId]
            data[Columns.Repository] = x[Columns.Repository]
            return data

        commit_series_per_repo = df.iloc[1:].apply(get_one_commit_series, axis=1)
        return pandas.concat(commit_series_per_repo.values)

    @staticmethod
    def extract_groups(df: pandas.DataFrame):
        data = df.loc[1:, [Columns.SeriesId, Columns.Repository,
                           Seriator.CLUSTER,  Seriator.SERIATED_IDX, "x_0", "x_1"]]
        return data
    
    @staticmethod
    def read_parquet(file_widget):
        f = io.BytesIO()
        f.write(file_widget.value)
        f.seek(0)
        return pd.read_parquet(f)
    
    
    def load_grouped_series(self, file_widget) -> pandas.DataFrame:
        """Load and deserialize the target parquet file containing encoded commit time series."""
        if file_widget.value is None:
            return None
        df = self.read_parquet(file_widget)
        start_date, end_date = df.loc[0, [Columns.SeriesId, Columns.Repository]].values.tolist()
        df.loc[:, Columns.Value] = df.loc[:, Columns.Value].astype(object)
        times = pandas.date_range(start_date, end_date, freq="D").values
        matrices = df.loc[1:, Columns.Value].map(deserialize_sparse_matrix).values.tolist()
        df.loc[:, Columns.Value] = [times] + matrices
        return df
    
    def read_parquet_files(self):
        grouped = self.load_grouped_series(self.grouped_file)
        embeddings = self.read_parquet(self.embeddings_file) if self.embeddings_file.value is not None else None
        commits = self.read_parquet(self.commits_file) if self.commits_file.value is not None else None
        return commits, embeddings, grouped
    
    @param.depends("load_parquet_btn.clicks", watch=True)
    def load_data(self):
        if self.load_parquet_btn.clicks == 0:
            return panel.pane.HTML("")
        dataframes = self.read_parquet_files()
        self.commits, self.embeddings, self.grouped = dataframes
        self._data_loaded = True
        return panel.pane.HTML(self.grouped.head().to_html() if self.grouped is not None
                               else "No data loaded")
    
    def preprocess_data(self):
        grouped_commits = self.extract_commits_per_repo(self.grouped)
        grouped_embeddings = self.extract_groups(self.grouped)
        commits_with_teams = self.commits.merge(self.embeddings,
                                         left_on="name",
                                         right_index=True, how="inner")
        commits_with_teams = commits_with_teams.drop(["x_0", "x_1", "type_x"], axis=1)
        commits_with_teams = commits_with_teams.rename(columns={"type_y": "type"})
        
        return commits_with_teams, self.embeddings, grouped_commits, grouped_embeddings

    def to_sql(self, *args, **kwargs):
        commits, embeddings, grouped_commits, grouped_embeddings = self.preprocess_data()
        grouped_commits.to_sql(self.GROUPED_COMMITS_TABLE, self.engine, *args, **kwargs)
        grouped_embeddings.to_sql(self.GROUPED_EMBEDDINGS_TABLE, self.engine, *args, **kwargs)
        commits.to_sql(self.COMMITS_TABLE, self.engine, *args, **kwargs)
        embeddings.to_sql(self.EMBEDDINGS_TABLE, self.engine, *args, **kwargs)
    
    @param.depends("to_sql_btn.clicks", watch=True)
    def to_parquet(self):
        if self._data_loaded:
            mkdwn = ("Created the following tables: \n")
            try:
                commits, embeddings, grouped_commits, grouped_embeddings = self.preprocess_data()
                grouped_commits.to_parquet(self.GROUPED_COMMITS_TABLE + ".parquet")
                mkdwn += "- {}\n".format(self.GROUPED_COMMITS_TABLE)
                grouped_embeddings.to_parquet(self.GROUPED_EMBEDDINGS_TABLE + ".parquet")
                mkdwn += "- {}\n".format(self.GROUPED_EMBEDDINGS_TABLE)
                commits.to_parquet(self.COMMITS_TABLE + ".parquet")
                mkdwn += "- {}\n".format(self.COMMITS_TABLE)
                embeddings.to_parquet(self.EMBEDDINGS_TABLE + ".parquet")
                mkdwn += "- {}\n".format(self.EMBEDDINGS_TABLE)
            except Exception as e:
                raise e
                pass
            return panel.pane.Str(mkdwn)
        else:
            return panel.pane.Str("")
        
    def panel(self):
        return panel.Column(panel.pane.Markdown("###Load eee-cts data into superset"),
                            panel.Row(panel.pane.Str("Embeddings    "), self.embeddings_file),
                            panel.Row(panel.pane.Str("Grouped Series"), self.grouped_file),
                            panel.Row(panel.pane.Str("Commit Series "), self.commits_file),
                            panel.Row(self.load_data),
                            panel.Row(self.to_parquet),
                            panel.Row(self.load_parquet_btn, self.to_sql_btn))

class WriteParquet(param.Parameterized):
    write_btn = widgets.Button(name="Write to DB")
    read_btn = widgets.Button(name="Read DB table")
    parquet_file = widgets.FileInput(name="Parquet file")
    table = widgets.TextInput(name="DB table")
    table_names = widgets.Select(name="DB tables")
    
    def __init__(self, engine):
        self.engine = engine
        self.df = None
        self.db_df = None
        
    
    @param.depends("parquet_file.value", watch=True)
    def load_parquet(self):
        f = io.BytesIO()
        f.write(self.file_input.value)
        f.seek(0)
        self.df = pd.read_parquet(f)
    
    @param.depends("datasource_btn.clicks", watch=True)
    def get_datasource(self):
        if self.data_source.value != "":
            sql = "SELECT * \nFROM {}\nLIMIT 10000".format(self.table.value)
            self.df = self.sql(sql)
        table = self.table.value if self.table.value != "" else "None"
        return panel.pane.Str("Selected Table {}".format(table))
    
    def sql(self, sql):
        df = pd.read_sql(sql, self.engine)
        return df
    
    def get_table_names(self):
        if self.engine is None:
            return
        names = pd.read_sql("SELECT tablename FROM pg_catalog.pg_tables;",
                            self.engine)
        self.table_names.options = names.values.flatten().tolist()
    
    @param.depends("write_btn.clicks", watch=True)
    def write_parquet(self):
        if self.write_btn.clicks > 0:
            self.load_parquet()
            self.df.to_sql(self.table.value, self.engine)
            return panel.pane.Str("Success")
        return panel.pane.Str("")
    
    @param.depends("read_btn.clicks", watch=True)
    def read_table(self):
        if self.read_btn.clicks > 0:
            sql = "SELECT * \nFROM {}\nLIMIT 10000".format(self.table_names.value)
            self.db_df = self.sql(sql)
            return panel.pane.HTML(self.db_df.head().to_html())
        return panel.pane.Str()
            
    
    
        
    def panel(self):
        return panel.Row(panel.Column(panel.Row(self.table_names),
                                      panel.Row(self.parquet_file),
                                      panel.Row(self.table),
                                      panel.Row(self.read_btn, self.write_btn),
                                      panel.Row(self.write_parquet)),
                         panel.Column(self.read_table))

class Dashboard(param.Parameterized):
    db_manager = DataManager()
    def __init__(self):
        
        self.parquet = WriteParquet(self.db_manager)
        self.cts_loader = CommitTimeSeriesPreprocessor(self.db_manager)
        self.parquet.write_btn.disabled = True
        self.parquet.read_btn.disabled = True
        
    @param.depends("db_manager.connect_btn.clicks", watch=True)
    def update_engine(self):
        if self.db_manager.connect_btn.clicks > 0:
            self.parquet.engine = self.db_manager.engine
            self.parquet.write_btn.disabled = False
            self.parquet.read_btn.disabled = False
            self.parquet.get_table_names()
            self.cts_loader.engine = engine
            
    
    def panel(self):
        return panel.Tabs(("Database", self.db_manager.panel()),
                          ("eee-cts", self.cts_loader.panel()),
                          ("Parquet2sql", panel.Column(self.parquet.panel(), self.update_engine)),
                         )
    
if __name__ == "__main__":
    panel.extension()
    dash = Dashboard()
    dash.panel().servable()
        

Overwriting main.py


Tabs
    [0] Column
        [0] Markdown(str)
        [1] Row
            [0] Column
                [0] TextInput(name='Database type', value='postgresql')
                [1] TextInput(name='Host', value='localhost')
                [2] LiteralInput(name='Port', value=5432)
            [1] Column
                [0] TextInput(name='Username', value='superset')
                [1] TextInput(name='Password', value='superset')
                [2] TextInput(name='Host', value='superset')
            [2] Column
                [0] ParamMethod(method)
                [1] Button(name='Connect to DB')
    [1] Column
        [0] Markdown(str)
        [1] Row
            [0] Str(str)
            [1] FileInput(name='Embeddings')
        [2] Row
            [0] Str(str)
            [1] FileInput(name='Grouped series')
        [3] Row
            [0] Str(str)
            [1] FileInput(name='Commits')
        [4] Row
            [0] ParamMethod(method)
        [5] Row
            [0] ParamMethod(method)
        [6] Row
            [0] Button(name='Load files')
            [1] Button(name='write to database.')
    [2] Column
        [0] Row
            [0] Column
                [0] Row
                    [0] Select(name='DB tables')
                [1] Row
                    [0] FileInput(name='Parquet file')
                [2] Row
                    [0] TextInput(name='DB table')
                [3] Row
                    [0] Button(disabled=True, name='Read DB table')
                    [1] Button(disabled=True, name='Write to DB')
                [4] Row
                    [0] ParamMethod(method)
            [1] Column
                [0] ParamMethod(method)
        [1] ParamMethod(method)

In [ ]:
class CommitTimeSeriesPreprocessor:

    EMBEDDINGS_TABLE = "developer_embeddings"
    COMMITS_TABLE = "commit_series"
    GROUPED_COMMITS_TABLE = "commits_per_repo"
    GROUPED_EMBEDDINGS_TABLE = "clusters_per_repo"

    def __init__(self, embeddings_path: str, commits_path: str, grouped_path: str, engine):
        from sqlalchemy import create_engine
        self.embeddings_path = embeddings_path
        self.commits_path = commits_path
        self.grouped_path = grouped_path
        self.embeddings = None
        self.grouped = None
        self.commits = None
        self.load_data()
        self.db_path = db_path
        self.engine = engine

    @staticmethod
    def extract_commits_per_repo(df):
        """
        Extract a DataFrame with columns ['date_ix', 'commits', 'name', 'repository'] \
        from grouped commit series data.
        """
        def get_one_commit_series(x):
            data = pandas.DataFrame(index=df.loc[0, Columns.Value],
                                    data=numpy.array(x[Columns.Value].todense()).flatten(),
                                    columns=[Columns.Value])
            data.index.name = Columns.Date
            data = data.reset_index()
            data = data[data[Columns.Value] > 0]
            data[Columns.SeriesId] = x[Columns.SeriesId]
            data[Columns.Repository] = x[Columns.Repository]
            return data

        commit_series_per_repo = df.iloc[1:].apply(get_one_commit_series, axis=1)
        return pandas.concat(commit_series_per_repo.values)

    @staticmethod
    def extract_groups(df: pandas.DataFrame):
        data = df.loc[1:, [Columns.SeriesId, Columns.Repository,
                           Seriator.CLUSTER,  Seriator.SERIATED_IDX, "x_0", "x_1"]]
        return data

    @staticmethod
    def read_parquet_files(commits_path: str, embeddings_path: str, grouped_path: str):
        grouped = DataExporter.load_grouped_series(path=grouped_path)
        embeddings = pandas.read_parquet(path=embeddings_path)
        commits = pandas.read_parquet(path=commits_path)
        return commits, embeddings, grouped

    def load_data(self):
        dataframes = self.read_parquet_files(commits_path=self.commits_path,
                                             embeddings_path=self.embeddings_path,
                                             grouped_path=self.grouped_path)
        self.commits, self.embeddings, self.grouped = dataframes
    
    def preprocess_data(self):
        grouped_commits = self.extract_commits_per_repo(self.grouped)
        grouped_embeddings = self.extract_groups(self.grouped)
        commits_with_teams = self.commits.merge(self.embeddings,
                                         left_on="name",
                                         right_index=True, how="inner")
        commits_with_teams = commits_with_teams.drop(["x_0", "x_1", "type_x"], axis=1)
        commits_with_teams = commits_with_teams.rename(columns={"type_y": "type"})
        
        return commits_with_teams, self.embeddings, grouped_commits, grouped_embeddings

    def to_sql(self, *args, **kwargs):
        commits, embeddings, grouped_commits, grouped_embeddings = self.preprocess_data()
        grouped_commits.to_sql(self.GROUPED_COMMITS_TABLE, self.engine, *args, **kwargs)
        grouped_embeddings.to_sql(self.GROUPED_EMBEDDINGS_TABLE, self.engine, *args, **kwargs)
        commits.to_sql(self.COMMITS_TABLE, self.engine, *args, **kwargs)
        embeddings.to_sql(self.EMBEDDINGS_TABLE, self.engine, *args, **kwargs)
        
    def to_parquet(self, *args, **kwargs):
        commits, embeddings, grouped_commits, grouped_embeddings = self.preprocess_data()
        grouped_commits.to_parquet(self.GROUPED_COMMITS_TABLE + ".parquet", *args, **kwargs)
        grouped_embeddings.to_parquet(self.GROUPED_EMBEDDINGS_TABLE + ".parquet", *args, **kwargs)
        commits.to_parquet(self.COMMITS_TABLE + ".parquet", *args, **kwargs)
        embeddings.to_parquet(self.EMBEDDINGS_TABLE + ".parquet", *args, **kwargs)